# Data Splitting and Image Registration

The preprocessing generally consists of the following basic steps:
1. Splitting the video into the before and after puncture phases
2. For each phase:
  1. Initial preprocessing which consists of some or all of the following
    1. Pseudo-flat field correction
    2. Coarse frame alignment
    3. Backgrounds Subtraction
    4. Copping
  2. Frame-by-frame image registration

In [ ]:
using DrWatson
@quickactivate()
using Interact, Queryverse
include(srcdir("reg.jl"));

In [ ]:
# You need to change directory to the directory in which the data resides
cd(datadir("example_video1"))

In [ ]:
"""
Visualize videos using a slider to move between frames.
"""
function visualize(frames)
    @manipulate throttle=0.1 for t=1:size(frames, timedim(frames))
        frames[time=t]
    end
end

We are going to have a global `force` option to forcibly recompute things.

In [ ]:
force = false;

## Loading and Splitting Video

We start out by loading the `video.avi` file, and visualize it to identify the frame at which the puncture happens.

In [ ]:
frames = loadframes("video.avi"; color=Gray{N0f16});

In [ ]:
visualize(frames)

If there is a puncture the video needs to be split into a "before" phase (the frames preceeding the puncture), and the "after" phase which begins some number of frames after the puncture until either the end of the video or some major event (e.g. a subsequent puncture or a bumped table). If you are processing the whole video uninterrupted then continue forward with the frames variable

We visualize these to make sure we are pleased with the split.

In [ ]:
# Your job is to choose the ranges below to trim out the puncture
before, after = frames[:,:,1:246], frames[:,:,297:end];

In [ ]:
visualize(before)

In [ ]:
visualize(after)

## Processing the frames before the puncture

In [ ]:
beforeregdir = joinpath("before", "registered");

In [ ]:
@time beforereg = before |>
    correctorload(beforeregdir; force=true, regularize=true, ffsubtract=true, cropvid=true);

In [ ]:
visualize(beforereg)

In [ ]:
size(beforereg)

In [ ]:
rm(beforeregdir; force=true, recursive=true)
mkpath(beforeregdir)
saveframes(beforeregdir, beforereg);

## Process the frames after the puncture

In [ ]:
afterregdir = joinpath("after", "registered");

In [ ]:
@time afterreg = after |>
    correctorload(afterregdir; force=true, regularize=true, ffsubtract=true, cropvid=true);

In [ ]:
visualize(afterreg)

In [ ]:
size(afterreg)

In [ ]:
rm(afterregdir; force=true, recursive=true)
mkpath(afterregdir)
saveframes(afterregdir, afterreg);